# Getting the data
TODO make work with day 2

Bushiroad uploads a list of all the players who participated in each of their higher level events, with each player's:
* handle
* placement
* \# of wins
* nation
* grade 3 in r.d.
* decklog id

This information is uploaded to a URL, with each page's HTML containing links to decklog, so we will need to:
* create a dataframe out of the information on the main page
* follow those decklog links to obtain more detailed data
* (in the future, look up individual card names for even more detailed data)
* add the more grainular data into the original dataframe
* store it all for future use in a central database

*todo note: As I work on this, I'm still deciding if each upload should include information on the particular event, and keep everything in one large table, or if the tables should be split up, so that information is kept intrisicly by which table the data was added too.*

The data from decklists doesn't fit too fell into a traditional database of rows and columns, but all the information can fit neatly into dictionaries, or JSON.
Each object we save to the database should contain 2 'levels' of data:
* the information from the original webpage at the base level at the base level
* a dictionary contaiing information about the deck list, which is nested dictionary, containing a dict each for the
    * main,
    * ride,
    * and g decks

Each 'layer' will be populated by a different pipleine.
Pipeline 1 will get data from the central page, placing it into the first dictionary,
and pipelie 2 will take the dekcklogs frome pipeline 1, obtain data from each link, using it to fill the second dictionary
A third pipeline will save this data to the database.

##### NOTE
Due to the structure of the main table we're scraping having special scripts for the ride deck g3's of the top 3 players, when using the second pipeline, the ride deck g3's name needs to be extracted, and as a key value pair to the first dicionary

## **Part** 1

# **NOTE** This script requires an input sring, 'EVENT' in order to function

### Obtain HTML for event, and extract to df

In [1]:

import pandas as pd

import requests
from bs4 import BeautifulSoup

# URL = "https://en.cf-vanguard.com/event/bcs2526/bcs2526-california/"
# EVENT = 'bcs2526-california'
BASE_URL = "https://en.cf-vanguard.com/event/bcs2526/"
EVENT = 'mexico'
url = f'{BASE_URL}{EVENT}'
soup = BeautifulSoup(requests.get(url).text)

In [2]:
def get_values(row):
    """This procederal function deals with the fact that the first 3 rows of
    the table have a slightly different structure than the rest.

    While I could write two functions, one for the first 3 rows, 
    and one for the rest, this just felt more simple and natural to write,
    allthough it's a bit tough to read.

    Our i counter being manual allows us to skip over the missing row
    for our top 3 players, while still keeping the incremental couting logic.

    Since the first 3 rows are a different size, we need to adjust the amount of values,
    so our data frame is happy. It ended up seeming simplest to just add `None` as we pass through
    """
    values=list()
    for i, item in enumerate(row):
        if i == 1: # This double conditional is true when the first 3 row's second element
            if item.div: # is a division, instead of text. we need to treat it different
                values.append("Champion") # we could extract the name, but it would be very complex
                values.append(None) # Add a none placeholder for our missing value
                continue # this continue statement seperates this special behaviour from the simple case

        values.append(row[i].text.strip())

    return values

In [3]:
rows = soup.table.find_all('tr')
# Remove header row, so all rows have td, and not th
rows.pop(0)

dataDict = dict()
for row in rows:
    # Decklog as the key
    key = row['data-deck-id']
    values = get_values(row.find_all('td'))
    values.append(key)
    values.append(None) # place holder

    dataDict[key] = values

In [4]:
df = pd.DataFrame(dataDict).transpose()
df = df.set_axis([
                    'rank',
                    'name',
                    'boss',
                    "wins",
                    'nation',
                    'decklog',
                    'deck'
                ], 
                axis=1)

df.head()

,rank,name,boss,wins,nation,decklog,deck
5LJTP,1st,Champion,None,10,Brandt Gate,5LJTP,None
6E32Z,2nd,Champion,None,8,Dark States,6E32Z,None
52YEH,3rd,Champion,None,9,Dragon Empire,52YEH,None
VRNF,4th,Aramis,"Sealed Blaze of Arbitration, Bavsargra Aksayya",9,Dragon Empire,VRNF,None
714ED,5th,Oscar Villalba,"Rewrite the Star, Vyrgilla",7,Dragon Empire,714ED,None


## Part 2 

### Obtain HTML for each deck's decklog, and add to df

Now, we need to settup a function to run on each decklog that we have in the dataframe

Not every decklog will have a G zone, and will thus contain a different number of 'row' divisions.
As far as standard decks go, there will be 7 rows if there is no g zone, and up one more to 8 with one.
The use of `None` for the G deck will help us skip over it without creating any errors.
These magic numbers for indexing below are a bit cringe, so I hope to come back and fix this later.

**NOTE: THIS WILL NEED UPDATING FOR PREMIUM DECKS DUE TO MAGIC NUMBER HACK**

In [5]:
def decklogToDict(soup):
    """
    TODO make boss work with premium
    
    Given the BeautifulSoup for a decklogs...
    """
    # Initialize the decks we return
    boss = None
    rideDeckDict = dict()
    mainDeckDict = dict()
    gDeckDict = dict()
    deckDict = {
        'RideDeck': rideDeckDict,
        'MainDeck': mainDeckDict,
        'GDeck': gDeckDict
    }

    # obtain data from soup
    rows =  soup.find_all('div', 'row')
    rideDeck = rows[5].find_all('div', 'card-controller')
    mainDeck = rows[6].find_all('div', 'card-controller')
    if len(rows) == 8:
        gDeck = rows[7].find_all('div', 'card-controller')
    else:
        gDeck = None

    # pair soup data with dict
    decks = {
        "RideDeck": rideDeck, 
        "MainDeck": mainDeck,
        "GDeck": gDeck
    }

    # extract data from the soup into the dictionary
    for deck in deckDict:
        if not decks[deck]: continue
        for card in decks[deck]:
            spans = card.find_all('span')
            namespan= str(spans[0])
            index1 = namespan.find(' : ') + 3 #trim whitespace
            index2 = namespan.find('"></span>')
            
            card_name = namespan[index1:index2]
            quant = int(spans[1].text)
        
            if card_name in deckDict[deck]:
                deckDict[deck][card_name] += quant
            else:
                boss = boss or card_name
                deckDict[deck][card_name] =  quant
            
    return deckDict, boss

In [6]:
#Credit Gemini for skeletron
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.expected_conditions import presence_of_element_located as present
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager

# Setup Chrome to run headless (without a visible window)
chrome_options = Options()
chrome_options.add_argument("--headless")
# Initialize the browser
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options) 
waiter = WebDriverWait(driver=driver, timeout=20)

# Go to each page
BASE_URL = "https://decklog-en.bushiroad.com/view/" 
code= "None"
for i, row in df.iterrows():
    # I've seen a few valid logs log make it through,
    # They usually just need a second try
    tries = 0
    while tries < 2:
        tries+=1
        try:
            if df.at[i, 'deck'] != None: continue # skip completed decks on re-run
            code = df.at[i, 'decklog']
            url = BASE_URL + str(code)
            driver.get(url)
            waiter.until(
                present((By.CLASS_NAME, "card-controller"))
            )
            html = driver.page_source
            deckSoup = BeautifulSoup(html)
            deckDict, boss = decklogToDict(deckSoup)
            df.at[i, 'deck'] = deckDict
            #TODO MAKE WORK WITH PREMIUM DECKS
            df.at[i, 'boss'] = boss
            
        except Exception as e:
           todo = df.deck.isnull().sum()
           print(f'{e}, \n{todo}, \n{code}')

# clean-up
driver.quit()

This bit ensures we finished getting each deck, in case we run into connection issues

In [7]:
# if None in df.deck: raise ValueError to preserve database authenticity
todo = df.deck.isnull().sum()
# if todo != 0: raise ValueError
print('',todo,'')


 0 


In [21]:
print(df.iloc[124].deck)

{'RideDeck': {'Sharp Bullet of Dust Storm, Thruston': 1, 'Venatiol, Mawuleizs': 1, 'Rockpick Dracokid': 1, 'Vyrgilla': 1, 'Energy Generator': 1}, 'MainDeck': {'Venatiol, Dhinbarada': 3, 'Sharp Bullet of Dust Storm, Thruston': 3, 'Sploosh Cutter Dragon': 4, 'Leidenschaft Dragon': 4, 'Wrought Iron Soldier Dragon, Indomitable Axe Dragon': 4, 'Venatiol, Mawuleizs': 4, 'Dancing Princess of Resonating Rock, Crushuna': 4, 'Sealed Blaze Dragon, Samupadru': 4, 'Dragritter, Metwalli': 4, 'Dragon Deity King of Resurgence, Dragveda': 1, 'Rogue Beast, Sunegosuri': 4, 'Jaggy Dagger Dragon': 3, 'Refreshing Wind Dragon, Dinobreeze': 4, 'Blaze War Monk, Sougyou': 4}, 'GDeck': {}}


In [ ]:

#!#!#!#!#
df = df[~df.deck.isnull()]

todo = df.deck.isnull().sum()
# if todo != 0: raise ValueError
print('',todo,'')

In [ ]:
### I've encountered some erros with weird decklogs that aren't correct. [5.97E+06, 2.62E+02]

## Part 3
### Upload the data

TODO FIX SECURITY ISSUE

In [33]:
from pymongo import MongoClient

# Connect to MongoDB
username = 'sjmichael17_db_user'
password = 'rVtL43eBjseB5XkS' # plz don't hack me bro ;-;
cluster_address = 'bcsproto.peazuyx.mongodb.net/?appName=BCSproto'

client = MongoClient(f'mongodb+srv://{username}:{password}@{cluster_address}')

db = client['JSONproto']
collection = db[EVENT]

cleanDF = df.reset_index().rename(columns={'index':'_id'})
collection.insert_many(cleanDF.to_dict(orient='records'))

print("Done")
client.close()

Done


In [ ]:
# ~~~~~~ BONUS FUNCTION ~~~~~~ #

In [31]:
def get_single_decklog(log):
    # Initialize the browser
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options) 
    waiter = WebDriverWait(driver=driver, timeout=30)
    
    # Go to each page
    BASE_URL = "https://decklog-en.bushiroad.com/view/" 
    # log= "3TW53 "
    url = f'{BASE_URL}{log}'
    
    driver.get(url)
    waiter.until(
        present((By.CLASS_NAME, "card-controller"))
    )
    html = driver.page_source
    deckSoup = BeautifulSoup(html)
    deckDict, boss = decklogToDict(deckSoup)
    
    print(str(deckDict).replace("'", '"'))
    
    # clean-up
    driver.quit()

In [ ]:
foo = [
    
]
for bar in foo:
    get_single_decklog(bar)
    print('(~~~~~~~~~~~~~~~~~~~~)')